In [ ]:

#@title Click play on the left once you have filled out the form!
token = "" #@param {type: "string"}
dataset_id = "" #@param {type: "string"}
sentiment_field = "" #@param {type: "string"}
label_field = "" #@param {type: "string"}
number_of_labels = 30 #@param {type: "integer"}

!pip install -q circlify
!pip install -q RelevanceAI-dev

from relevanceai import Client
client = Client(token=token)
ds = client.Dataset(dataset_id)

import circlify
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
facets = ds.facets([label_field], page_size=number_of_labels)
labels = [x['value'] for x in facets['results'][label_field]]
freq = [x['frequency'] for x in facets['results'][label_field]]

circles = circlify.circlify(
    freq, 
    show_enclosure=False, 
    target_enclosure=circlify.Circle(x=0, y=0, r=1)
)
# Create just a figure and only one subplot
fig, ax = plt.subplots(figsize=(10,10))

# Title
ax.set_title('Sentiment Wordcloud Bubbles')

# Remove axes
ax.axis('off')

# Find axis boundaries
lim = max(
    max(
        abs(circle.x) + circle.r,
        abs(circle.y) + circle.r,
    )
    for circle in circles
)
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

# print circles
for circle, label in tqdm(zip(circles, labels)):
    x, y, r = circle
    sent_field = "avg_sentiment"
    sentiment = ds.aggregate(
        metrics=[{"name": sent_field, 
                  "field": sentiment_field, 
                  "agg": "avg"}],
        filters=ds[label_field] == label
    )
    result = sentiment['results'][0][sent_field]
    if result < 0:
        color = 'red'
    else:
        color = 'green'
    ax.add_patch(plt.Circle((x, y), r, alpha=0.2, linewidth=2, fill=True, color=color))
    plt.annotate(
          label,
          (x,y ) ,
          va='center',
          ha='center'
     )
